In [126]:
from abc import abstractmethod
from typing import Protocol, TypeVar

T = TypeVar("T")


class AbelianGroupOperation(Protocol[T]):
    @abstractmethod
    def add(self, a: T, b: T) -> T:
        raise NotImplementedError

    @abstractmethod
    def neg(self, a: T) -> T:
        raise NotImplementedError

    @abstractmethod
    def identity(self) -> T:
        raise NotImplementedError

    def is_commutative(self, a: T, b: T) -> bool:
        test = self.add(a, b) == self.add(b, a)
        if not test:
            print(
                f"Failed commutativity assertion: {self.add(a, b)} == {self.add(b, a)}"
            )

        return test

    def is_associative(self, a: T, b: T, c: T) -> bool:
        test = self.add(self.add(a, b), c) == self.add(a, self.add(b, c))
        if not test:
            print(
                f"Failed associativity assertion: {self.add(self.add(a, b), c)} == {self.add(a, self.add(b, c))}"
            )

        return test

    def has_identity(self, a: T) -> bool:
        identity = self.identity()
        test = self.add(a, identity) == a and self.add(identity, a) == a
        if not test:
            print(
                f"Failed identity assertion: {self.add(a, identity)} == {self.add(identity, a)}"
            )

        return test

    def has_inverse(self, a: T) -> bool:
        identity = self.identity()
        inv_a = self.neg(a)
        test = self.add(a, inv_a) == identity and self.add(inv_a, a) == identity
        if not test:
            print(
                f"Failed inverse assertion: {self.add(a, inv_a)} == {self.add(inv_a, a)}"
            )

        return test


class IntegerAddition(AbelianGroupOperation[int]):
    def add(self, a: int, b: int) -> int:
        return a + b

    def neg(self, a: int) -> int:
        return -a

    def identity(self) -> int:
        return 0

In [127]:
a: int = 5
b: int = 6

integer_addition_group = IntegerAddition()

print(f"{a} + {b} = {integer_addition_group.add(a, b)}\n")

print(integer_addition_group.is_commutative(a, b))
print(integer_addition_group.is_associative(a, b, integer_addition_group.add(a, b)))
print(integer_addition_group.has_identity(a))
print(integer_addition_group.has_inverse(a))

5 + 6 = 11

True
True
True
True


In [128]:
from typing import Callable, Generic, Iterator, List, Optional, Union


class Stream(Generic[T]):
    timestamp: int
    inner: List[T]
    group_op: AbelianGroupOperation[T]

    def __init__(self, group_op: AbelianGroupOperation[T]) -> None:
        self.inner = []
        self.group_op = group_op
        self.timestamp = -1

    def send(self, element: T) -> None:
        self.inner.append(element)
        self.timestamp += 1

    def group(self) -> AbelianGroupOperation[T]:
        return self.group_op

    def current_time(self) -> int:
        return self.timestamp

    def __iter__(self) -> Iterator[T]:
        return self.inner.__iter__()

    def __repr__(self) -> str:
        return self.inner.__repr__()

    def __getitem__(self, timestamp: int) -> T:
        if timestamp <= self.current_time() and timestamp >= 0:
            return self.inner.__getitem__(timestamp)

        return self.group().identity()

    def latest(self) -> T:
        return self[self.current_time()]

    def __eq__(self, other) -> bool:
        if not isinstance(other, Stream):
            return False

        self_timestamp = self.current_time()
        other_timestamp = other.current_time()

        if self_timestamp != other_timestamp:
            largest = max(self_timestamp, other_timestamp)

            for timestamp in range(largest):
                self_val = self[timestamp]
                other_val = other[timestamp]

                if self_val != other_val:
                    return False

            return True

        return self.inner == other.inner


StreamReference = Callable[[], Stream[T]]


class StreamHandle(Generic[T]):
    ref: StreamReference[T]

    def __init__(self, stream_reference: StreamReference[T]) -> None:
        self.ref = stream_reference

    def get(self) -> Stream:
        return self.ref()


class Operator(Protocol):
    @abstractmethod
    def step(self) -> bool:
        raise NotImplementedError

    @abstractmethod
    def output_handle(self) -> StreamHandle:
        raise NotImplementedError


R = TypeVar("R", covariant=True)

TR = Union[T, R]


class UnaryOperator(Operator, Protocol[T, R]):
    input_stream_handle: StreamHandle[T]
    output_stream_handle: StreamHandle[TR]

    def __init__(
        self,
        stream_handle: Optional[StreamHandle[T]],
        output_stream_group: Optional[AbelianGroupOperation[TR]],
    ) -> None:
        if stream_handle is not None:
            self.set_input(stream_handle, output_stream_group)

    def set_input(
        self,
        stream_handle: StreamHandle[T],
        output_stream_group: Optional[AbelianGroupOperation[TR]],
    ) -> None:
        self.input_stream_handle = stream_handle
        if output_stream_group is not None:
            output = Stream(output_stream_group)

            self.output_stream_handle = StreamHandle(lambda: output)
        else:
            output = Stream(self.input_a().group())

            self.output_stream_handle = StreamHandle(lambda: output)

    def output(self) -> Stream[TR]:
        return self.output_stream_handle.get()

    def input_a(self) -> Stream[T]:
        return self.input_stream_handle.get()

    def output_handle(self) -> StreamHandle[TR]:
        handle = StreamHandle(lambda: self.output())

        return handle


class BinaryOperator(Operator, Protocol[T, R]):
    input_stream_handle_a: StreamHandle[T]
    input_stream_handle_b: StreamHandle[T]
    output_stream_handle: StreamHandle[TR]

    def __init__(
        self,
        stream_a: Optional[StreamHandle[T]],
        stream_b: Optional[StreamHandle[T]],
        output_stream_group: Optional[AbelianGroupOperation[TR]],
    ) -> None:
        if stream_a is not None:
            self.set_input_a(stream_a)

        if stream_b is not None:
            self.set_input_b(stream_b)

        if output_stream_group is not None:
            output = Stream(output_stream_group)

            self.set_output_stream(StreamHandle(lambda: output))

    def set_input_a(self, stream_handle_a: StreamHandle[T]) -> None:
        self.input_stream_handle_a = stream_handle_a
        output = Stream(self.input_a().group())

        self.set_output_stream(StreamHandle(lambda: output))

    def set_input_b(self, stream_handle_b: StreamHandle[T]) -> None:
        self.input_stream_handle_b = stream_handle_b

    def set_output_stream(self, output_stream_handle: StreamHandle[TR]) -> None:
        self.output_stream_handle = output_stream_handle

    def output(self) -> Stream[TR]:
        return self.output_stream_handle.get()

    def input_a(self) -> Stream[T]:
        return self.input_stream_handle_a.get()

    def input_b(self) -> Stream[T]:
        return self.input_stream_handle_b.get()

    def output_handle(self) -> StreamHandle[TR]:
        handle = StreamHandle(lambda: self.output())

        return handle

In [129]:
from typing import Any, Callable


class Delay(UnaryOperator[T, T]):
    def __init__(self, stream: Optional[StreamHandle[T]]) -> None:
        super().__init__(stream, None)

    def step(self) -> bool:
        output_timestamp = self.output().current_time()

        delayed_value = self.input_a()[output_timestamp]

        self.output().send(delayed_value)

        return True


F1 = TypeVar("F1", bound=Callable[[Any], Any])


class Lifted1(UnaryOperator[T, R], Generic[T, R, F1]):
    f1: F1

    def __init__(
        self,
        stream: Optional[StreamHandle[T]],
        f1: F1,
        output_stream_group: Optional[AbelianGroupOperation[TR]],
    ):
        self.f1 = f1
        super().__init__(stream, output_stream_group)

    def step(self) -> bool:
        output_timestamp = self.output().current_time()

        self.output().send(self.f1(self.input_a()[output_timestamp + 1]))

        return True


class LiftedGroupNegate(Lifted1[T, T, Callable[[T], T]]):
    def __init__(self, stream: StreamHandle[T]):
        super().__init__(stream, lambda x: stream.get().group().neg(x), None)


F2 = TypeVar("F2", bound=Callable[[Any, Any], Any])


class Lifted2(BinaryOperator[T, R], Generic[T, R, F2]):
    def __init__(
        self,
        stream_a: Optional[StreamHandle[T]],
        stream_b: Optional[StreamHandle[T]],
        f2: F2,
        output_stream_group: Optional[AbelianGroupOperation[TR]],
    ) -> None:
        self.f2 = f2
        super().__init__(stream_a, stream_b, output_stream_group)

    def step(self) -> bool:
        output_timestamp = self.output().current_time()

        a = self.input_a()[output_timestamp + 1]
        b = self.input_b()[output_timestamp + 1]

        application = self.f2(a, b)
        self.output().send(application)

        return True


class LiftedGroupAdd(Lifted2[T, T, Callable[[T, T], T]]):
    def __init__(self, stream_a: StreamHandle[T], stream_b: Optional[StreamHandle[T]]):
        super().__init__(
            stream_a,
            stream_b,
            lambda x, y: stream_a.get().group().add(x, y),
            None,
        )


class Differentiate(UnaryOperator[T, T]):
    delayed_stream: Delay[T]
    delayed_negated_stream: LiftedGroupNegate[T]
    differentiation_stream: LiftedGroupAdd[T]

    def __init__(self, stream: StreamHandle[T]) -> None:
        self.input_stream_handle = stream
        self.delayed_stream = Delay(self.input_stream_handle)
        self.delayed_negated_stream = LiftedGroupNegate(
            self.delayed_stream.output_handle()
        )
        self.differentiation_stream = LiftedGroupAdd(
            self.input_stream_handle, self.delayed_negated_stream.output_handle()
        )
        self.output_stream_handle = self.differentiation_stream.output_handle()

    def step(self) -> bool:
        self.delayed_stream.step()
        self.delayed_negated_stream.step()
        return self.differentiation_stream.step()


class Integrate(UnaryOperator[T, T]):
    delayed_stream: Delay[T]
    integration_stream: LiftedGroupAdd[T]

    def __init__(self, stream: StreamHandle[T]) -> None:
        self.input_stream_handle = stream
        self.integration_stream = LiftedGroupAdd(self.input_stream_handle, None)
        self.delayed_stream = Delay(self.integration_stream.output_handle())
        self.integration_stream.set_input_b(self.delayed_stream.output_handle())

        self.output_stream_handle = self.integration_stream.output_handle()

    def step(self) -> bool:
        self.delayed_stream.step()
        return self.integration_stream.step()

In [130]:
def create_integer_identity_stream(to: int) -> Stream[int]:
    stream = Stream(IntegerAddition())
    for i in range(to):
        stream.send(i)

    return stream


def step_n_times(operator: Operator, n: int):
    for _ in range(n):
        operator.step()


def saturate(operator: Operator):
    while operator.step():
        pass

In [131]:
n = 10
integer_identity_stream = create_integer_identity_stream(n)
zero_to_ten = StreamHandle(lambda: integer_identity_stream)
print(f"Stream of integers from 0 to {n}: \n{zero_to_ten.get()}\n")
steps_to_catchup = zero_to_ten.get().current_time() + 1

Stream of integers from 0 to 10: 
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]



In [132]:
delayed_zero_to_ten = Delay(zero_to_ten)
step_n_times(delayed_zero_to_ten, steps_to_catchup)
print(f"Delayed stream of integers: \n{delayed_zero_to_ten.output()}\n")

Delayed stream of integers: 
[0, 0, 1, 2, 3, 4, 5, 6, 7, 8]



In [133]:
diff_zero_to_ten = Differentiate(zero_to_ten)
step_n_times(diff_zero_to_ten, steps_to_catchup)
print(f"Diff stream of integers: \n{diff_zero_to_ten.output()}\n")

Diff stream of integers: 
[0, 1, 1, 1, 1, 1, 1, 1, 1, 1]



In [134]:
int_zero_to_ten = Integrate(zero_to_ten)
step_n_times(int_zero_to_ten, steps_to_catchup)
print(f"Integration stream of integers: \n{int_zero_to_ten.output()}\n")

Integration stream of integers: 
[0, 1, 3, 6, 10, 15, 21, 28, 36, 45]



In [135]:
int_diff_zero_to_ten = Integrate(diff_zero_to_ten.output_handle())
step_n_times(int_diff_zero_to_ten, steps_to_catchup)
print(f"Integrated diff stream of integers: \n{int_diff_zero_to_ten.output()}\n")

Integrated diff stream of integers: 
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]



In [136]:
from typing import Dict, Generic, Iterable, Tuple


class ZSet(Generic[T]):
    inner: Dict[T, int]

    def __init__(self, values: Dict[T, int]) -> None:
        self.inner = values

    def items(self) -> Iterable[Tuple[T, int]]:
        return self.inner.items()

    def __repr__(self) -> str:
        return self.inner.__repr__()

    def __eq__(self, other) -> bool:
        if not isinstance(other, ZSet):
            return False

        return self.inner == other.inner

    def __getitem__(self, item: T) -> int:
        if item not in self.inner:
            return 0

        return self.inner[item]

    def __contains__(self, item: T) -> bool:
        return self.inner.__contains__(item)


class ZSetAddition(AbelianGroupOperation[ZSet[T]]):
    def add(self, a: ZSet[T], b: ZSet[T]) -> ZSet[T]:
        c = {k: v for k, v in a.inner.items() if v != 0}
        for k, v in b.inner.items():
            if k in c:
                new_weight = c[k] + v
                if new_weight != 0:
                    c[k] = new_weight
                else:
                    del c[k]
            else:
                c[k] = v

        return ZSet(c)

    def neg(self, a: ZSet[T]) -> ZSet[T]:
        return ZSet({k: v * -1 for k, v in a.inner.items()})

    def identity(self) -> ZSet[T]:
        return ZSet({})


A: ZSet[str] = ZSet({"apple": 1, "orange": 3})
B: ZSet[str] = ZSet({"apple": 3, "banana": 2})

zset_addition_group = ZSetAddition()

C = zset_addition_group.add(A, B)
print(f"{A} + {B} = {C}\n")

D = zset_addition_group.neg(A)
print(f"neg({A}) = {D}\n")

print(zset_addition_group.is_commutative(A, B))
print(zset_addition_group.is_associative(A, B, zset_addition_group.add(A, B)))
print(zset_addition_group.has_identity(A))
print(zset_addition_group.has_inverse(A))

{'apple': 1, 'orange': 3} + {'apple': 3, 'banana': 2} = {'apple': 4, 'orange': 3, 'banana': 2}

neg({'apple': 1, 'orange': 3}) = {'apple': -1, 'orange': -3}

True
True
True
True


In [137]:
class StreamAddition(AbelianGroupOperation[Stream[T]]):
    group: AbelianGroupOperation[T]

    def __init__(self, group: AbelianGroupOperation[T]) -> None:
        self.group = group

    def add(self, a: Stream[T], b: Stream[T]) -> Stream[T]:
        handle_a = StreamHandle(lambda: a)
        handle_b = StreamHandle(lambda: b)

        lifted_group_add = LiftedGroupAdd(handle_a, handle_b)
        times = max(a.current_time(), b.current_time()) + 1
        step_n_times(lifted_group_add, times)

        return lifted_group_add.output()

    def neg(self, a: Stream[T]) -> Stream[T]:
        handle_a = StreamHandle(lambda: a)
        lifted_group_neg = LiftedGroupNegate(handle_a)
        times = a.current_time() + 1
        step_n_times(lifted_group_neg, times)

        return lifted_group_neg.output()

    def identity(self) -> Stream[T]:
        identity_stream = Stream(self.group)

        return identity_stream

In [138]:
A = ZSet({"apple": 1, "orange": 3})
B = ZSet({"apple": -2, "orange": 3})
C = ZSet({"apple": 2, "bananas": -2})

zset_stream = Stream(ZSetAddition())
zset_stream.send(A)
zset_stream.send(B)
zset_stream.send(C)


zset_stream_addition_group = StreamAddition(zset_addition_group)
zset_stream_sum = zset_stream_addition_group.add(zset_stream, zset_stream)

print(f"zset_stream: {zset_stream}\n")
print(f"zset_stream + zset_stream:\n{zset_stream_sum}\n")
print(zset_stream_addition_group.is_commutative(zset_stream, zset_stream))
print(
    zset_stream_addition_group.is_associative(zset_stream, zset_stream, zset_stream_sum)
)
print(zset_stream_addition_group.has_identity(zset_stream))
print(zset_stream_addition_group.has_inverse(zset_stream))

zset_stream: [{'apple': 1, 'orange': 3}, {'apple': -2, 'orange': 3}, {'apple': 2, 'bananas': -2}]

zset_stream + zset_stream:
[{'apple': 2, 'orange': 6}, {'apple': -4, 'orange': 6}, {'apple': 4, 'bananas': -4}]

True
True
True
True


In [139]:
JoinCmp = Callable[[T, T], bool]
PostJoinProjection = Callable[[T, T], T]


def join[T](
    left_zset: ZSet[T],
    right_zset: ZSet[T],
    p: JoinCmp,
    f: PostJoinProjection,
) -> ZSet[T]:
    output = {}
    for left_value, left_weight in left_zset.items():
        for right_value, right_weight in right_zset.items():
            if p(left_value, right_value):
                projected_value = f(left_value, right_value)
                new_weight = left_weight * right_weight

                if projected_value in output:
                    output[projected_value] += new_weight
                else:
                    output[projected_value] = new_weight

    return ZSet(output)

In [140]:
import math

some_zset: ZSet[int] = ZSet({1: 1, 2: 1, 3: 1, 4: 1})
other_zset: ZSet[int] = ZSet({4: 1, 9: 1})

some_join_cmp = lambda left, right: left == math.isqrt(right)
some_post_join_projection = lambda left, right: left

print(join(some_zset, other_zset, some_join_cmp, some_post_join_projection))

{2: 1, 3: 1}


In [141]:
class LiftedJoin(Lifted2[ZSet[T], ZSet[T], Callable[[ZSet[T], ZSet[T]], ZSet[T]]]):
    def __init__(
        self,
        stream_a: Optional[StreamHandle[ZSet[T]]],
        stream_b: Optional[StreamHandle[ZSet[T]]],
        p: JoinCmp,
        f: PostJoinProjection,
    ):
        super().__init__(stream_a, stream_b, lambda x, y: join(x, y, p, f), None)

In [142]:
def step_n_times_and_return(operator: Operator, n: int) -> Stream:
    step_n_times(operator, n)

    return operator.output_handle().get()

In [143]:
some_zset_stream = Stream(ZSetAddition())
some_zset_stream.send(some_zset)

other_zset_stream = Stream(ZSetAddition())
other_zset_stream.send(other_zset)

lifted_join_stream = LiftedJoin(
    StreamHandle(lambda: some_zset_stream),
    StreamHandle(lambda: other_zset_stream),
    some_join_cmp,
    some_post_join_projection,
)

print(step_n_times_and_return(lifted_join_stream, 1))

[{2: 1, 3: 1}]


In [144]:
class LiftedLiftedJoin(
    Lifted2[
        Stream[ZSet[T]],
        Stream[ZSet[T]],
        Callable[[Stream[ZSet], Stream[ZSet]], Stream[ZSet[T]]],
    ]
):
    def __init__(
        self,
        stream_a: Optional[StreamHandle[Stream[ZSet[T]]]],
        stream_b: Optional[StreamHandle[Stream[ZSet[T]]]],
        p: JoinCmp,
        f: PostJoinProjection,
    ):
        super().__init__(
            stream_a,
            stream_b,
            lambda x, y: step_n_times_and_return(
                LiftedJoin(StreamHandle(lambda: x), StreamHandle(lambda: y), p, f),
                max(x.current_time(), y.current_time()) + 1,
            ),
            None,
        )

In [145]:
some_stream_of_zset_streams = Stream(StreamAddition(ZSetAddition()))
some_stream_of_zset_streams.send(some_zset_stream)

other_stream_of_zset_streams = Stream(StreamAddition(ZSetAddition()))
other_stream_of_zset_streams.send(other_zset_stream)

lifted_lifted_join_stream = LiftedLiftedJoin(
    StreamHandle(lambda: some_stream_of_zset_streams),
    StreamHandle(lambda: other_stream_of_zset_streams),
    some_join_cmp,
    some_post_join_projection,
)

print(step_n_times_and_return(lifted_lifted_join_stream, 1))

[[{2: 1, 3: 1}]]


In [146]:
UnaryOp = TypeVar("UnaryOp", bound=UnaryOperator)


class LiftedDelay(Lifted1[Stream[T], Stream[T], Callable[[Stream[T]], Stream[T]]]):
    def __init__(self, stream: StreamHandle[Stream[T]]):
        super().__init__(
            stream,
            lambda s: step_n_times_and_return(
                Delay(StreamHandle(lambda: s)), s.current_time() + 1
            ),
            None,
        )


class LiftedIntegrate(Lifted1[Stream[T], Stream[T], Callable[[Stream[T]], Stream[T]]]):
    def __init__(self, stream: StreamHandle[Stream[T]]):
        super().__init__(
            stream,
            lambda s: step_n_times_and_return(
                Integrate(StreamHandle(lambda: s)), s.current_time() + 1
            ),
            None,
        )


class LiftedDifferentiate(
    Lifted1[Stream[T], Stream[T], Callable[[Stream[T]], Stream[T]]]
):
    def __init__(self, stream: StreamHandle[Stream[T]]):
        super().__init__(
            stream,
            lambda s: step_n_times_and_return(
                Differentiate(StreamHandle(lambda: s)), s.current_time() + 1
            ),
            None,
        )

In [147]:
class LiftedMod2(Lifted1[int, int, Callable[[int], int]]):
    def __init__(self, stream: StreamHandle[int]):
        super().__init__(stream, lambda i: i % 2, None)


class LiftedLiftedMod2(
    Lifted1[Stream[int], Stream[int], Callable[[Stream[int]], Stream[int]]]
):
    def __init__(self, stream: StreamHandle[Stream[int]]):
        super().__init__(
            stream,
            lambda s: step_n_times_and_return(
                LiftedMod2(StreamHandle(lambda: s)), s.current_time() + 1
            ),
            None,
        )


example = Stream(StreamAddition(IntegerAddition()))
for t1 in range(4):
    s = Stream(IntegerAddition())

    for t0 in range(4):
        s.send(t0 + (2 * t1))

    example.send(s)
print(f"example:\n{example}\n")
example_n_times = example.current_time() + 1
example_handle = StreamHandle(lambda: example)

mod2 = LiftedLiftedMod2(example_handle)
step_n_times(mod2, example_n_times)
print(f"↑↑mod2(example):\n{mod2.output()}\n")

int_example = Integrate(example_handle)
step_n_times(int_example, example_n_times)
print(f"∫(example):\n{int_example.output()}\n")

lift_int_example = LiftedIntegrate(example_handle)
step_n_times(lift_int_example, example_n_times)
print(f"↑∫(example):\n{lift_int_example.output()}\n")

diff_example = Differentiate(example_handle)
step_n_times(diff_example, example_n_times)
print(f"D(example):\n{diff_example.output()}\n")

lift_diff_example = LiftedDifferentiate(example_handle)
step_n_times(lift_diff_example, example_n_times)
print(f"↑D(example):\n{lift_diff_example.output()}\n")

delayed_example = Delay(example_handle)
step_n_times(delayed_example, example_n_times)
print(f"z^-1(example):\n{delayed_example.output()}\n")

lifted_delayed_example = LiftedDelay(example_handle)
step_n_times(lifted_delayed_example, example_n_times)
print(f"↑z^-1(example):\n{lifted_delayed_example.output()}\n")

example:
[[0, 1, 2, 3], [2, 3, 4, 5], [4, 5, 6, 7], [6, 7, 8, 9]]

↑↑mod2(example):
[[0, 1, 0, 1], [0, 1, 0, 1], [0, 1, 0, 1], [0, 1, 0, 1]]

∫(example):
[[0, 1, 2, 3], [2, 4, 6, 8], [6, 9, 12, 15], [12, 16, 20, 24]]

↑∫(example):
[[0, 1, 3, 6], [2, 5, 9, 14], [4, 9, 15, 22], [6, 13, 21, 30]]

D(example):
[[0, 1, 2, 3], [2, 2, 2, 2], [2, 2, 2, 2], [2, 2, 2, 2]]

↑D(example):
[[0, 1, 1, 1], [2, 1, 1, 1], [4, 1, 1, 1], [6, 1, 1, 1]]

z^-1(example):
[[], [0, 1, 2, 3], [2, 3, 4, 5], [4, 5, 6, 7]]

↑z^-1(example):
[[0, 0, 1, 2], [0, 2, 3, 4], [0, 4, 5, 6], [0, 6, 7, 8]]



In [148]:
class LiftedLiftedDeltaJoin(BinaryOperator[Stream[ZSet[T]], Stream[ZSet[T]]]):
    p = JoinCmp[T]
    f = PostJoinProjection[T]

    integrated_stream_a: Integrate[Stream[ZSet[T]]]
    delayed_integrated_stream_a: Delay[Stream[ZSet[T]]]
    lift_integrated_stream_a: LiftedIntegrate[ZSet[T]]
    integrated_lift_integrated_stream_a: Integrate[Stream[ZSet[T]]]

    integrated_stream_b: Integrate[Stream[ZSet[T]]]
    delayed_integrated_stream_b: Delay[Stream[ZSet[T]]]
    lift_integrated_stream_b: LiftedIntegrate[ZSet[T]]
    integrated_lift_integrated_stream_b: Integrate[Stream[ZSet[T]]]
    lift_delayed_integrated_lift_integrated_stream_b: LiftedDelay[ZSet[T]]
    lift_delayed_lift_integrated_stream_b: LiftedDelay[ZSet[T]]

    join_1: LiftedLiftedJoin[T]
    join_2: LiftedLiftedJoin[T]
    join_3: LiftedLiftedJoin[T]
    join_4: LiftedLiftedJoin[T]

    sum_one: LiftedGroupAdd[Stream[ZSet[T]]]
    sum_two: LiftedGroupAdd[Stream[ZSet[T]]]
    sum_three: LiftedGroupAdd[Stream[ZSet[T]]]

    output_stream: Stream[ZSet[T]]

    def set_input_a(self, stream_handle_a: StreamHandle[Stream[ZSet[T]]]) -> None:
        self.input_stream_handle_a = stream_handle_a
        self.integrated_stream_a = Integrate(self.input_stream_handle_a)
        self.delayed_integrated_stream_a = Delay(
            self.integrated_stream_a.output_handle()
        )

        self.lift_integrated_stream_a = LiftedIntegrate(self.input_stream_handle_a)
        self.integrated_lift_integrated_stream_a = Integrate(
            self.lift_integrated_stream_a.output_handle()
        )

    def set_input_b(self, stream_handle_b: StreamHandle[Stream[ZSet[T]]]) -> None:
        self.input_stream_handle_b = stream_handle_b
        self.integrated_stream_b = Integrate(self.input_stream_handle_b)
        self.delayed_integrated_stream_b = Delay(
            self.integrated_stream_b.output_handle()
        )

        self.lift_integrated_stream_b = LiftedIntegrate(self.input_stream_handle_b)
        self.integrated_lift_integrated_stream_b = Integrate(
            self.lift_integrated_stream_b.output_handle()
        )
        self.lift_delayed_integrated_lift_integrated_stream_b = LiftedDelay(
            self.integrated_lift_integrated_stream_b.output_handle()
        )
        self.lift_delayed_lift_integrated_stream_b = LiftedDelay(
            self.lift_integrated_stream_b.output_handle()
        )

        self.join_1 = LiftedLiftedJoin(
            self.delayed_integrated_stream_a.output_handle(),
            self.lift_delayed_lift_integrated_stream_b.output_handle(),
            self.p,
            self.f,
        )
        self.join_2 = LiftedLiftedJoin(
            self.integrated_lift_integrated_stream_a.output_handle(),
            self.input_stream_handle_b,
            self.p,
            self.f,
        )
        self.join_3 = LiftedLiftedJoin(
            self.lift_integrated_stream_a.output_handle(),
            self.delayed_integrated_stream_b.output_handle(),
            self.p,
            self.f,
        )
        self.join_4 = LiftedLiftedJoin(
            self.input_stream_handle_a,
            self.lift_delayed_integrated_lift_integrated_stream_b.output_handle(),
            self.p,
            self.f,
        )

        self.sum_one = LiftedGroupAdd(
            self.join_1.output_handle(), self.join_2.output_handle()
        )
        self.sum_two = LiftedGroupAdd(
            self.sum_one.output_handle(), self.join_3.output_handle()
        )
        self.sum_three = LiftedGroupAdd(
            self.sum_two.output_handle(), self.join_4.output_handle()
        )

        self.set_output_stream(self.sum_three.output_handle())

    def __init__(
        self,
        diff_stream_a: Optional[StreamHandle[Stream[ZSet[T]]]],
        diff_stream_b: Optional[StreamHandle[Stream[ZSet[T]]]],
        p: JoinCmp,
        f: PostJoinProjection,
    ):
        self.p = p
        self.f = f

        if diff_stream_a is not None:
            self.set_input_a(diff_stream_a)

        if diff_stream_b is not None:
            self.set_input_b(diff_stream_b)

    def step(self) -> bool:
        self.integrated_stream_a.step()
        self.delayed_integrated_stream_a.step()
        self.lift_integrated_stream_a.step()
        self.integrated_lift_integrated_stream_a.step()

        self.integrated_stream_b.step()
        self.delayed_integrated_stream_b.step()
        self.lift_integrated_stream_b.step()
        self.integrated_lift_integrated_stream_b.step()
        self.lift_delayed_integrated_lift_integrated_stream_b.step()
        self.lift_delayed_lift_integrated_stream_b.step()

        self.join_1.step()
        self.join_2.step()
        self.join_3.step()
        self.join_4.step()

        self.sum_one.step()
        self.sum_two.step()
        self.sum_three.step()

        return True

In [149]:
lifted_lifted_delta_join_stream = LiftedLiftedDeltaJoin(
    StreamHandle(lambda: some_stream_of_zset_streams),
    StreamHandle(lambda: other_stream_of_zset_streams),
    some_join_cmp,
    some_post_join_projection,
)

print(step_n_times_and_return(lifted_lifted_delta_join_stream, 1))

[[{2: 1, 3: 1}]]


In [150]:
def H[T](diff: ZSet[T], integrated_state: ZSet[T]) -> ZSet[T]:
    distincted_diff = {}
    for k, v in diff.items():
        current_k_latest_diff_weight = v

        if k in integrated_state:
            current_k_latest_delayed_state_weight = integrated_state[k]
            coalesced_weight = (
                current_k_latest_diff_weight + current_k_latest_delayed_state_weight
            )

            if current_k_latest_delayed_state_weight > 0 and coalesced_weight <= 0:
                distincted_diff[k] = -1

                continue

            if current_k_latest_delayed_state_weight <= 0 and coalesced_weight > 0:
                distincted_diff[k] = 1
        else:
            if v > 0:
                distincted_diff[k] = 1

    return ZSet(distincted_diff)

In [151]:
some_zset_update_one = ZSet({1: -1, 6: -1})

H(some_zset_update_one, some_zset)

{1: -1}

In [152]:
class LiftedH(Lifted2[ZSet[T], ZSet[T], Callable[[ZSet[T], ZSet[T]], ZSet[T]]]):
    def __init__(
        self,
        diff_stream_a: StreamHandle[ZSet[T]],
        integrated_stream_a: StreamHandle[ZSet[T]],
    ):
        super().__init__(diff_stream_a, integrated_stream_a, H, None)

In [153]:
some_zset_update_one_stream = Stream(ZSetAddition())
some_zset_update_one_stream.send(some_zset_update_one)

step_n_times_and_return(
    LiftedH(
        StreamHandle(lambda: some_zset_update_one_stream),
        StreamHandle(lambda: some_zset_stream),
    ),
    1,
)

[{1: -1}]

In [154]:
class LiftedLiftedH(
    Lifted2[
        Stream[ZSet[T]],
        Stream[ZSet[T]],
        Callable[[Stream[ZSet[T]], Stream[ZSet[T]]], Stream[ZSet[T]]],
    ]
):
    def __init__(
        self,
        integrated_diff_stream_a: StreamHandle[Stream[ZSet[T]]],
        lifted_delayed_lifted_integrated_stream_a: StreamHandle[Stream[ZSet[T]]],
    ):
        super().__init__(
            integrated_diff_stream_a,
            lifted_delayed_lifted_integrated_stream_a,
            lambda x, y: step_n_times_and_return(
                LiftedH(StreamHandle(lambda: x), StreamHandle(lambda: y)),
                max(x.current_time(), y.current_time()) + 1,
            ),
            None,
        )

In [155]:
some_zset_update_stream_stream = Stream(StreamAddition(ZSetAddition()))
some_zset_update_stream_stream.send(some_zset_update_one_stream)

step_n_times_and_return(
    LiftedLiftedH(
        StreamHandle(lambda: some_zset_update_stream_stream),
        StreamHandle(lambda: some_stream_of_zset_streams),
    ),
    1,
)

[[{1: -1}]]

In [156]:
class DeltaLiftedDeltaLiftedDistinct(UnaryOperator[Stream[ZSet[T]], Stream[ZSet[T]]]):
    integrated_diff_stream_a: Integrate[Stream[ZSet[T]]]
    lift_integrated_diff_stream_a: LiftedIntegrate[ZSet[T]]
    lift_delay_lift_integrated_diff_stream_a: LiftedDelay[ZSet[T]]
    lift_lift_H: LiftedLiftedH[T]
    diff_lift_lift_H: Differentiate[Stream[ZSet[T]]]

    def set_input(
        self,
        stream_handle: StreamHandle[Stream[ZSet[T]]],
        output_stream_group: Optional[AbelianGroupOperation[Stream[ZSet[T]]]],
    ) -> None:
        self._input_stream_a = stream_handle
        self.integrated_diff_stream_a = Integrate(self._input_stream_a)
        self.lift_integrated_diff_stream_a = LiftedIntegrate(
            self.integrated_diff_stream_a.output_handle()
        )
        self.lift_delay_lift_integrated_diff_stream_a = LiftedDelay(
            self.lift_integrated_diff_stream_a.output_handle()
        )
        self.lift_lift_H = LiftedLiftedH(
            self.integrated_diff_stream_a.output_handle(),
            self.lift_delay_lift_integrated_diff_stream_a.output_handle(),
        )
        self.diff_lift_lift_H = Differentiate(self.lift_lift_H.output_handle())
        self.output_stream_handle = self.diff_lift_lift_H.output_handle()

    def __init__(self, diff_stream_a: Optional[StreamHandle[Stream[ZSet[T]]]]):
        super().__init__(diff_stream_a, None)

    def step(self) -> bool:
        self.integrated_diff_stream_a.step()
        self.lift_integrated_diff_stream_a.step()
        self.lift_delay_lift_integrated_diff_stream_a.step()
        self.lift_lift_H.step()
        return self.diff_lift_lift_H.step()

In [157]:
stream_of_zset_streams = Stream(StreamAddition(ZSetAddition()))
stream_of_zset_streams.send(some_zset_stream)

delta_lift_delta_lift_distinct = DeltaLiftedDeltaLiftedDistinct(
    StreamHandle(lambda: stream_of_zset_streams)
)
delta_lift_delta_lift_distinct.step()
print(delta_lift_delta_lift_distinct.output())

stream_of_zset_streams.send(some_zset_update_one_stream)
delta_lift_delta_lift_distinct.step()
print(delta_lift_delta_lift_distinct.output())

[[{1: 1, 2: 1, 3: 1, 4: 1}]]
[[{1: 1, 2: 1, 3: 1, 4: 1}], [{1: -1}]]


In [211]:
def stream_introduction[T](value: T, group: AbelianGroupOperation[T]) -> Stream[T]:
    output_stream = Stream(group)
    output_stream.send(value)

    return output_stream


def stream_elimination[T](stream: Stream[T]) -> T:
    output_value = stream.group().identity()
    for value in stream:
        output_value = stream.group().add(output_value, value)

    return output_value


class LiftedStreamIntroduction(Lifted1[T, Stream[T], Callable[[T], Stream[T]]]):
    def __init__(self, stream: StreamHandle[T]) -> None:
        super().__init__(
            stream,
            lambda x: stream_introduction(x, stream.get().group()),
            StreamAddition(stream.get().group()),
        )


class LiftedStreamElimination(Lifted1[Stream[T], T, Callable[[Stream[T]], T]]):
    def __init__(self, stream: StreamHandle[Stream[T]]) -> None:
        super().__init__(stream, lambda x: stream_elimination(x), None)

In [212]:
print(zset_stream)

stream_of_zset_stream = stream_introduction(zset_stream, zset_stream_addition_group)
print(stream_of_zset_stream)

print(stream_elimination(zset_stream))

print(stream_elimination(stream_of_zset_stream))

[{'apple': 1, 'orange': 3}, {'apple': -2, 'orange': 3}, {'apple': 2, 'bananas': -2}]
[[{'apple': 1, 'orange': 3}, {'apple': -2, 'orange': 3}, {'apple': 2, 'bananas': -2}]]
{'apple': 1, 'orange': 6, 'bananas': -2}
[{'apple': 1, 'orange': 3}, {'apple': -2, 'orange': 3}, {'apple': 2, 'bananas': -2}]


In [213]:
type GraphZSet = ZSet[tuple[int, int]]


class IncrementalGraphReachability(UnaryOperator[GraphZSet, GraphZSet]):
    delta_input: LiftedStreamIntroduction[GraphZSet]
    join: LiftedLiftedDeltaJoin[GraphZSet]
    delta_input_join_sum: LiftedGroupAdd[Stream[GraphZSet]]
    distinct: DeltaLiftedDeltaLiftedDistinct[GraphZSet]
    delayed_distinct: Delay[Stream[GraphZSet]]
    flattened_output: LiftedStreamElimination[GraphZSet]

    def __init__(self, stream: StreamHandle[GraphZSet]):
        self._input_stream = stream

        self.delta_input = LiftedStreamIntroduction(self._input_stream)

        self.join = LiftedLiftedDeltaJoin(
            None,
            None,
            lambda left, right: left[1] == right[0],
            lambda left, right: (left[0], right[1]),
        )
        self.delta_input_join_sum = LiftedGroupAdd(
            self.delta_input.output_handle(), self.join.output_handle()
        )
        self.distinct = DeltaLiftedDeltaLiftedDistinct(
            self.delta_input_join_sum.output_handle()
        )
        self.delayed_distinct = Delay(self.distinct.output_handle())
        self.join.set_input_a(self.delayed_distinct.output_handle())
        self.join.set_input_b(self.delta_input.output_handle())

        self.flattened_output = LiftedStreamElimination(self.distinct.output_handle())
        self.output_stream_handle = self.flattened_output.output_handle()

    def step(self) -> bool:
        self.delta_input.step()
        self.delayed_distinct.step()
        self.join.step()
        self.delta_input_join_sum.step()
        self.distinct.step()
        self.flattened_output.step()

        return True

In [214]:
import itertools as it

initial_graph: GraphZSet = ZSet(
    {(fst, snd): 1 for fst, snd in it.product(range(5), range(5)) if snd - 1 == fst}
)


def step_until_fixpoint(operator: Operator):
    out = operator.output_handle()
    out_identity = out.get().group().group.identity()
    while True:
        operator.step()
        latest = out.get().latest()
        if latest == out_identity:
            break

In [215]:
graph_stream = Stream(ZSetAddition())
graph_stream.send(initial_graph)
graph_stream_handle = StreamHandle(lambda: graph_stream)
reachability = IncrementalGraphReachability(graph_stream_handle)
print(f"Graph:\n{graph_stream}\n")
step_until_fixpoint(reachability)
print(f"Reachability Graph:\n{reachability.output_handle().get()}\n")

Graph:
[{(0, 1): 1, (1, 2): 1, (2, 3): 1, (3, 4): 1}]

Reachability Graph:
[{(0, 1): 1, (1, 2): 1, (2, 3): 1, (3, 4): 1}, {(0, 2): 1, (1, 3): 1, (2, 4): 1}, {(0, 3): 1, (1, 4): 1}, {(0, 4): 1}, {}]



In [229]:
import typing


def load_graph(file_path: str) -> GraphZSet:
    out = []

    with open(file_path, mode="r") as file:
        lines = file.readlines()
        out = {
            edge: 1
            for edge in map(
                lambda line: tuple(map(lambda node: int(node), line.split()[0:2])),
                lines,
            )
        }

    return typing.cast(GraphZSet, ZSet(out))

In [230]:
dense_graph_zset = load_graph("data/graph1000.txt")

In [231]:
%%time

dense_graph_stream = Stream(ZSetAddition())
dense_graph_stream.send(dense_graph_zset)
dense_graph_stream_handle = StreamHandle(lambda: dense_graph_stream)

dense_graph_reachability = IncrementalGraphReachability(dense_graph_stream_handle)
step_until_fixpoint(dense_graph_reachability)

CPU times: user 694 ms, sys: 10.9 ms, total: 705 ms
Wall time: 713 ms


In [232]:
import functools as func

result = func.reduce(
    lambda a, b: zset_addition_group.add(a, b), dense_graph_reachability.output()
)

# DBSP Rust outputs 11532 tuples
len(result.inner)

11532

In [235]:
sparse_graph_zset = load_graph("data/graph10000.txt")

In [236]:
%%time

sparse_graph_stream = Stream(ZSetAddition())
sparse_graph_stream.send(sparse_graph_zset)
sparse_graph_stream_handle = StreamHandle(lambda: sparse_graph_stream)

sparse_graph_reachability = IncrementalGraphReachability(sparse_graph_stream_handle)
step_until_fixpoint(sparse_graph_reachability)

CPU times: user 1min 38s, sys: 278 ms, total: 1min 38s
Wall time: 1min 40s


In [237]:
result = func.reduce(
    lambda a, b: zset_addition_group.add(a, b), sparse_graph_reachability.output()
)

# DBSP Rust outputs 262144 tuples
len(result.inner)

262144